In [ ]:
%load_ext autoreload
%autoreload 2

# Planet multi-target classification

## Data

In [ ]:
#export
from nb_005 import *
import pandas as pd
import csv
from collections import OrderedDict

In [ ]:
PATH = Path('data/planet')
TRAIN = PATH/'train-jpg'
label_csv = PATH/'train_v2.csv'
bs = 64
norm,denorm = normalize_funcs(tensor([0.3080, 0.3355, 0.2945]),
                              tensor([0.0015, 0.0010, 0.0011]))

In [ ]:
# norm,denorm = normalize_funcs(*imagenet_stats)

In [ ]:
df = pd.read_csv(label_csv, header=0)
df.head()

In [ ]:
df.iloc[:,1] = list(csv.reader(df.iloc[:,1], delimiter=' '))
df.head()

In [ ]:
#export
def uniqueify(x): return list(OrderedDict.fromkeys(x).keys())

In [ ]:
classes = uniqueify(np.concatenate(df.tags))
' '.join(classes)

In [ ]:
class FilesMultiDataset(LabelDataset):
    def __init__(self, fns, labels, classes=None):
        self.classes = ifnone(classes, uniqueify(np.concatenate(labels)))
        self.class2idx = {v:k for k,v in enumerate(self.classes)}
        self.x = np.array(fns)
        self.y = [np.array([self.class2idx[o] for o in l], dtype=np.int64)
                  for l in labels]

    def encode(self, x):
        res = np.zeros((self.c,), np.float32)
        res[x] = 1.
        return res
    
    def get_labels(self, idx): return [self.classes[i] for i in self.y[idx]]
    def __getitem__(self,i): return open_image(self.x[i]), self.encode(self.y[i])
    
    @classmethod
    def from_random_split(cls, fns, labels, test_pct, classes=None):
        train,valid = random_split(test_pct, fns, labels)
        train_ds = cls(*train)
        return train_ds,FilesMultiDataset(*valid, classes=train_ds.classes)

In [ ]:
def get_path(x): return f'{TRAIN}/' + x + '.jpg'

In [ ]:
tfms = get_transforms(do_flip=True, flip_vert=True, max_lighting=0.1)

In [ ]:
np.random.seed(42)
train_ds,valid_ds = FilesMultiDataset.from_random_split(
    get_path(df.image_name), df.tags, 0.1)
valid_ds[1][0].brightness(0.75).contrast(1.5).show(
    title=valid_ds.get_labels(1), figsize=(6,6))

In [ ]:
valid_ds[1][1]

## Model

In [ ]:
data = DataBunch.create(*transform_datasets(train_ds, valid_ds, tfms, size=128),
                        bs=bs, num_workers=12, tfms=norm)

In [ ]:
x,y = next(iter(data.valid_dl))
x,y = x.data,y.data

In [ ]:
accuracy??

In [ ]:
learn = ConvLearner(data, tvm.resnet34, 2, wd=1e-2)#, metrics=accuracy)
learn.loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr=0.2

In [ ]:
learn.fit_one_cycle(1, slice(lr), pct_start=0.05)

In [ ]:
learn.save('0')

In [ ]:
learn.load('0')

In [ ]:
m=learn.model

In [ ]:
with torch.no_grad():
    m.eval()
    print(m(x).sigmoid())

In [ ]:
with torch.no_grad():
    m.train()
    print(m(x).sigmoid())

In [ ]:
with torch.no_grad():
    m.train()
    print(learn.loss_fn(m(x), y))

## Fin